# Monte-Carlo For Menzel 2015 Econometrica

In [1]:
using LinearAlgebra, Statistics, ForwardDiff, JuMP, LinearAlgebra, Distributions, Random
using Optim
using Optim: converged, maximum, maximizer, minimizer, iterations 

# The Following Function Calculates the systematic parts of the Pay-Offs

In [2]:
function makeindex(X_women,X_men,θ_w)
    nw,k=size(X_women)
    nm,k=size(X_men)
    
    tw_own=θ_w[1:k]
    tw_oth=θ_w[k+1:2*k]
    tw_diff=θ_w[2*k+1:3*k]
    tw_dist=θ_w[3*k+1:4*k]
    
    Xb = reshape((repeat(X_women,nm,1)-reshape(repeat(X_men',nw,1),k,nm*nw)').^2*tw_dist,nw,nm);
    
    Xb = Xb + repeat(X_women*tw_own,1,nm) + repeat(tw_oth',nw,1)*X_men';
    Xb = Xb + repeat(X_women*tw_diff,1,nm) - repeat(tw_diff',nw,1)*X_men';
    
    return Xb
end
    

makeindex (generic function with 1 method)

# The Gale-Shapley: Proposer = U 

In [3]:
function Gale_Shapley(U,V)
    nw,nm=size(U)
    U_temp=U
    nmax=10*nw*nm
    
    for i in 1:nmax
        Prop = Int.((U_temp.==repeat(max.(findmax(U_temp,dims=2)[1],0),1,nm)))
        Rej = Prop.*(Int.((Prop.*V .< max.(0,repeat(findmax(Prop.*V,dims=1)[1],nw,1)))))
        U_temp[Rej.== Prop].=-1;
        
        if sum(sum(Rej))==0
            break
        end
        
    end
    
   return μ= Int.((U_temp.==repeat(max.(findmax(U_temp,dims=2)[1],0),1,nm)))
    
end
    

Gale_Shapley (generic function with 1 method)

# The Loglikelihood Function

In [8]:
function loglik2(θ::Vector,μ::AbstractArray,X::AbstractArray,Z::AbstractArray)
    
    if θ[1]>=0 && θ[2]>=0 && θ[3]>=0 && θ[1]<=1 && θ[2]<=1 && θ[3]<=2
    
        nw=length(X);
        nm=length(Z);    
        βw=[θ[1:2]...;0;0;0;0;0;θ[3]...]
        βm=βw;
    
        tol=1e-8
        maxiter=100
        Γold=zeros(4)
        i=0
    
    
        while norm(Ψ_fp1(βw,Γold,X,Z)-Γold)>tol && i<=10000
        Γold=Ψ_fp1(βw,Γold,X,Z)
        i=i+1
        end
   
        Γ=Ψ_fp1(βw,Γold,X,Z)      
    
    
        Γ_w_mat = log.((1 .+ Γ[1].*(1 .-X[:,2]) + Γ[2].*X[:,2]));
        Γ_m_mat = log.((1 .+ Γ[3].*(1 .-Z[:,2]) + Γ[4].*Z[:,2]));
    
        U_star = makeindex(X,Z,βw);
        V_star = makeindex(Z,X,βm)';
   
        W_star = μ.*(U_star + V_star);
    
    
        mar_w = sum(μ,dims=2);
        mar_m = sum(μ,dims=1)';
    
        LL = -(2*sum(sum(W_star)) - sum((1 .+mar_w).*Γ_w_mat) - sum((1 .+mar_m).*Γ_m_mat));
        return LL
    else
        return 1e8
    end
end




loglik2 (generic function with 1 method)

# Fixed Point Constraint

In [6]:
function Ψ_fp1(θ::Vector,Γ_0::Vector,X::AbstractArray,Z::AbstractArray)
    
        
    nw = length(X);
    nm = length(Z);
    k = length(θ);
    β_w = θ[1:k];
    β_m = θ[1:k];
    
    
    Γ_w_mat = 1 .+ Γ_0[1].*(1 .-X[:,2]) + Γ_0[2].*X[:,2];
    Γ_m_mat = 1 .+ Γ_0[3].*(1 .-Z[:,2]) + Γ_0[4].*Z[:,2];
    
    Xw = [1 0;1 1];
    
    U_star = makeindex(Xw,Z,β_w);
    V_star = makeindex(Z,Xw,β_m)';
    W_star_w = U_star + V_star;
    
    Zm = [1 0;1 1];
    
    
    U_star = makeindex(X,Zm,β_w);
    V_star = makeindex(Zm,X,β_m)';
    W_star_m = U_star + V_star;
    
    Ψ_w = mean((exp.(W_star_w))' ./repeat(Γ_m_mat,1,2),dims=1);
    Ψ_m = mean(exp.(W_star_m)./repeat(Γ_w_mat,1,2),dims=1);
    Ψ = [Ψ_w'..., Ψ_m'...]
    
    #Ψ_ineq = zeros(4,1);
    return Ψ
    
end

Ψ_fp1 (generic function with 1 method)

# MonteCarlo

In [9]:
        
Random.seed!(1234)



beta_sim=zeros(3,20)

for b in 1:20

        n=20
        alpha=0.5
        n2 =Int(n/2)
        X = repeat([1.0 0.0;1.0 1.0],n2,1)
        Z = repeat([1.0 0.0;1.0 1.0],n2,1)

        beta_w = [1;0;0;0;0;0;0;1];
        beta_m = [1;0;0;0;0;0;0;1];
        
        beta_w = [alpha;0.5;0;0;0;0;0;1];
        beta_m = [alpha;0.5;0;0;0;0;0;1];
       
        
        U_star = makeindex(X,Z,beta_w);
        V_star = makeindex(Z,X,beta_m)'

        η=-log.(-log.(rand(n,n)));
        ζ=-log.(-log.(rand(n,n)))
        
        J=Int(round(sqrt(n)))
        η₀=repeat(findmax(-log.(-log.(rand(J,n))),dims=1)[1]',1,n)
        ζ₀=repeat(findmax(-log.(-log.(rand(J,n))),dims=1)[1],n,1)

        η=η-η₀
        ζ=ζ-ζ₀
        
        U=U_star+η
        V=V_star+ζ

        μ=Gale_Shapley(U,V)
        
    
        

        #k = length(beta_w);
        
       #theta_0 = [beta_w...,beta_m...,zeros(4)...];
        #theta_0 = [0.5 .*ones(20)...]

        
        #thetalambda_0=[theta_0...,0.5 .*ones(32)...]
        logtmp(x)=loglik2(x,μ,X,Z);  
        function gradlog!(G,x)
            tmp= y->ForwardDiff.gradient(logtmp,y)
            G=tmp(x)
        end
        


        lower = zeros(3)
        upper= [Inf;Inf;Inf]#Inf;Inf;Inf;Inf;Inf]
        
        results = Optim.optimize(logtmp,ones(3),NelderMead())
        println("minimum = $(results.minimum) with argmin = $(results.minimizer[1:3]) in $(results.iterations) iterations")  
        beta_sim[:,b]=results.minimizer
        
end  

a=mean(beta_sim,dims=2)       

minimum = 41.97508332896412 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 50.21108992375649 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 55.538152604761656 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 58.42828456156438 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 50.21108992375648 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 74.64547919937232 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 52.86521528576684 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 55.65615590215785 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 58.428284561564396 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 47.538152604761656 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 66.4282845615644 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 58.192277966772025 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 52.865215285766844 with argmin = [1.0, 1.0, 1.0] in 88 iterations
minimum = 44.76602394

3×1 Array{Float64,2}:
 0.9999999999999999
 0.9999999999999999
 1.0               